In [54]:
from pathlib import Path
from llama_index.readers.file import PDFReader
from llama_index.readers.file import PyMuPDFReader
import os
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext
from llama_index.llms.openai import OpenAI
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes
from llama_index.core import Document
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core import VectorStoreIndex
from llama_index.llms.ollama import Ollama
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.node_parser import (
    HierarchicalNodeParser,
    SentenceSplitter,
)
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, ServiceContext, StorageContext
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from llama_index.llms.groq import Groq


In [65]:
from llama_index.core import Settings
Settings.llm = Groq(model="llama3-8b-8192", api_key=" ")
llm = Groq(model="llama3-8b-8192", api_key=" ")

In [57]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
    )


In [58]:
import csv
import pandas as pd
file_path = 'pdfsQna.csv'
df = pd.read_csv(file_path)
queries = df['Question'].tolist()


In [59]:
index = VectorStoreIndex.from_documents(
    documents, 
    embed_model=embed_model,
)


In [60]:
query_engine = index.as_query_engine(llm=llm)
hyde = HyDEQueryTransform(include_original=True,llm=llm)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)

In [61]:
def hyderetriver(queries,query_engine):
    result=[]
    file_path = 'pdfsQna.csv'
    df = pd.read_csv(file_path)
    for i, query in enumerate(queries): 
        print(query)
        ground_truth = df.iloc[i]['GroundTruth']  
        response = hyde_query_engine.query(query)
        query_bundle = hyde(query)
        hyde_doc = query_bundle.embedding_strs
        result.append({'question': query, 'answer': str(response),'contexts':hyde_doc, 'ground_truths': ground_truth})
    return result

In [66]:
hyderesult=hyderetriver(queries,query_engine)

What vulnerabilities has the COVID-19 pandemic revealed in the health care system and public health infrastructure?
How has the COVID-19 pandemic affected the biomedical and health research enterprises?
What challenges and lessons learned are discussed in the COVID 19 paper?
How did the COVID-19 pandemic highlight the necessity for researchers to share data?
What is the current status of research for COVID 19  described in the paper?
How did public-private collaborations contribute to vaccine development during the COVID-19 pandemic?
What strategies did the National Institutes of Health (NIH) adopt to accelerate the development of therapeutic interventions, vaccines, and diagnostics?
What priorities are outlined in The Science Agenda for COVID-19 established by the Centers for Disease Control and Prevention (CDC)?
How did research funders, including non-profit entities, contribute to the pandemic response?
What efforts did the Patient-Centered Outcomes Research Institute (PCORI) undert